<a href="https://colab.research.google.com/github/leonardooliveiraoficial/mba-data-projects/blob/main/svm_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de Crédito**

> Classificação com SVM Linear e Seleção de Atributos (RFE)

---
## **1. Coleta de dados**

```
Instalação/Importação de bibliotecas
```

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE

```
Leitura do CSV
```

In [ ]:
CSV_PATH = "/content/clientes_inadimplencia.csv"
RANDOM_STATE = 42
TEST_SIZE = 0.25
N_FEATURES_RFE = 5

```
Carregar o dataset
```

In [ ]:
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(
        f"Arquivo não encontrado em '{CSV_PATH}'. "
        "Envie o 'clientes_inadimplencia.csv' para o Colab (ícone de upload) "
        "ou ajuste o caminho em CSV_PATH."
    )

df = pd.read_csv(CSV_PATH)

print("Dataset carregado com sucesso!")
print("Dimensão:", df.shape)
print("\nPrimeiras linhas:")
print(df.head())

Dataset carregado com sucesso!
Dimensão: (500, 16)

Primeiras linhas:
   renda_mensal  idade  divida_total  limite_cartao  saldo_conta  \
0   5609.434160     39  24501.800746    4996.125702  5954.787330   
1   2920.031788     37   6835.015844   13050.758629  4013.166308   
2   6500.902392     54    121.316748    6407.070486   376.393668   
3   6881.129433     25  15792.278568   11134.618437  4342.456023   
4   1097.929623     30  22446.705811    8203.959878  1450.696287   

   gastos_online  gastos_supermercado  gastos_saude  gastos_lazer  \
0     716.403835          1953.301872    373.653443    469.671793   
1     363.796509           917.934059    929.770883      0.000000   
2    1765.556173           256.542653    929.123093   1442.263289   
3    1495.378147          2213.657642   1042.561279   2959.788055   
4     827.688828          2312.064993    968.478338    743.324844   

   numero_compras  tempo_emprego_anos  valor_investimentos     patrimonio  \
0               6            

```
Validar coluna alvo
```

In [ ]:
if "inadimplente" not in df.columns:
    raise ValueError("A coluna alvo 'inadimplente' não foi encontrada no dataset.")

```
Separar em treino (75%) e teste (25%)
```

In [ ]:
X = df.drop(columns=["inadimplente"])
y = df["inadimplente"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

```
Padronizar as variáveis
```

In [ ]:
scaler_full = StandardScaler()
X_train_scaled = scaler_full.fit_transform(X_train)
X_test_scaled = scaler_full.transform(X_test)

```
Treinar SVM Linear com todas as variáveis e calcular acurácia
```

In [ ]:
svm_full = SVC(kernel="linear", random_state=RANDOM_STATE)
svm_full.fit(X_train_scaled, y_train)
y_pred_full = svm_full.predict(X_test_scaled)
acc_full = accuracy_score(y_test, y_pred_full)

print("\n📊 Resultado com TODAS as variáveis:")
print(f"Acurácia (teste): {acc_full:.3f}")


📊 Resultado com TODAS as variáveis:
Acurácia (teste): 0.808


```
Aplicar RFE para selecionar 5 variáveis
```

In [ ]:
rfe = RFE(estimator=SVC(kernel="linear", random_state=RANDOM_STATE), n_features_to_select=N_FEATURES_RFE)
rfe.fit(X_train_scaled, y_train)
selected_mask = rfe.support_
selected_features = X.columns[selected_mask].tolist()

print("\n🔎 Variáveis selecionadas pelo RFE:")

for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")


🔎 Variáveis selecionadas pelo RFE:
1. divida_total
2. gastos_online
3. gastos_saude
4. gastos_lazer
5. tempo_emprego_anos


```
Re-treinar o modelo somente com as 5 variáveis selecionadas
```

In [ ]:
X_train_rfe = X_train[selected_features]
X_test_rfe = X_test[selected_features]

scaler_rfe = StandardScaler()
X_train_rfe_scaled = scaler_rfe.fit_transform(X_train_rfe)
X_test_rfe_scaled = scaler_rfe.transform(X_test_rfe)

svm_rfe = SVC(kernel="linear", random_state=RANDOM_STATE)
svm_rfe.fit(X_train_rfe_scaled, y_train)
y_pred_rfe = svm_rfe.predict(X_test_rfe_scaled)
acc_rfe = accuracy_score(y_test, y_pred_rfe)

print("\n📊 Resultado com APENAS as variáveis selecionadas (RFE):")
print(f"Acurácia (teste): {acc_rfe:.3f}")


📊 Resultado com APENAS as variáveis selecionadas (RFE):
Acurácia (teste): 0.808


```
Conclusão automática básica
```

In [ ]:
print("\n🧠 Conclusão:")
delta = acc_full - acc_rfe
if abs(delta) < 0.01:
    print(
        "O modelo com menos variáveis manteve desempenho muito semelhante ao modelo completo. "
        "Isso sugere que coletar todas as variáveis pode não ser necessário para boa performance."
    )
elif delta > 0:
    print(
        f"O modelo reduzido perdeu {delta:.3f} de acurácia em relação ao completo. "
        "Avalie o custo/benefício de coletar todas as variáveis versus simplicidade."
    )
else:
    print(
        f"O modelo reduzido GANHOU {-delta:.3f} de acurácia. "
        "Isso indica que a seleção de atributos ajudou a remover ruído e simplificar o modelo."
    )


🧠 Conclusão:
O modelo com menos variáveis manteve desempenho muito semelhante ao modelo completo. Isso sugere que coletar todas as variáveis pode não ser necessário para boa performance.
